## Extract CBC's and UMI's

In [1]:
%%bash
#!/usr/bin/env bash

scripts="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix"

bc_reads="/Users/manuel/Desktop/bowtie_strategy/SRR6750059_2_1mio.fastq"
output="/Users/manuel/Desktop/bowtie_strategy"
umi_out_name="extracted_umis"
cbc_out_name="extracted_cbcs"

python3 $scripts/extract_cbc_umi.py \
    --bc_reads $bc_reads \
    --out_dir $output \
    --cbc_out_name $cbc_out_name \
    --umi_out_name $umi_out_name

/Users/manuel/Desktop/bowtie_strategy/SRR6750059_2_1mio.fastq
/Users/manuel/Desktop/bowtie_strategy
extracted_cbcs
extracted_umis
Filename: /Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix/extract_cbc_umi.py

Line #    Mem usage    Increment   Line Contents
    31     38.3 MiB     38.3 MiB   @profile
    32                             def extract_bc_umi_to_fastq(path_to_reads, out_dir,  bc_out_filename, umi_out_filename):
    33                                 '''
    34                                 Extracts the barcodes and umis from bc_read.fastq file. Then saves them in two separate .fastq files
    35                                 :param path_to_reads:
    36                                 :param out_dir:
    37                                 :param bc_out_filename:
    38                                 :param umi_out_filename:
    39                                 :return:
    40                                 '''
    41     38.

## Create bowtie_index of bc_combinations

In [4]:
%%bash
#!/usr/bin/env bash

genome=/Users/manuel/Desktop/bowtie_strategy/barcode_combinations.fasta
out=/Users/manuel/Desktop/bowtie_strategy/bc_index_files/bc_index

bowtie2-build \
     --threads 4 \
     $genome \
     $out

Settings:
  Output files: "/Users/manuel/Desktop/bowtie_strategy/bc_index_files/bc_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 16
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /Users/manuel/Desktop/bowtie_strategy/barcode_combinations.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 663552
Using parameters --bmax 497664 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructin

Building a SMALL index


## Align bc_reads to bc_combinations

In [7]:
%%bash
#!/usr/bin/env bash

bowtie2_index="/Users/manuel/Desktop/bowtie_strategy/bc_index_files/bc_index"
genomic_reads="/Users/manuel/Desktop/bowtie_strategy/extracted_cbcs.fastq"
output="/Users/manuel/Desktop/bowtie_strategy/aligned_bcs.sam"

bowtie2 \
     -p 4 \
     -x $bowtie2_index \
     -U $genomic_reads \
     -S $output


1000000 reads; of these:
  1000000 (100.00%) were unpaired; of these:
    688349 (68.83%) aligned 0 times
    311651 (31.17%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
31.17% overall alignment rate


## Select CBCs, UMIs and genes

In [8]:
%%bash
#!/usr/bin/env bash

scripts="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix"

path_to_cbcs_sam="/Users/manuel/Desktop/bowtie_strategy/aligned_bcs.sam"
path_to_bc_comb="/Users/manuel/Desktop/bowtie_strategy/barcode_combinations.fasta"
path_to_umis_fastq="/Users/manuel/Desktop/bowtie_strategy/extracted_umis.fastq"
path_to_gen_fastq="/Users/manuel/Desktop/bowtie_strategy/SRR6750059_1_1mio.fastq"
out_dir="/Users/manuel/Desktop/bowtie_strategy"
out_filename_bcs="selected_barcodes"
out_filename_umis="selected_umis"
out_filename_gen="selected_gens"

python3 $scripts/select_bcs_umis_genes.py \
     --sam_in $path_to_cbcs_sam \
     --cbc_comb $path_to_bc_comb \
     --umi_in $path_to_umis_fastq \
     --gen_reads $path_to_gen_fastq \
     --out_dir $out_dir \
     --cbc_out_name $out_filename_bcs \
     --umi_out_name $out_filename_umis \
     --gene_out_name $out_filename_gen

## Align selected genomic reads to reference genome (on cluster)

In [ ]:
%%bash
#!/usr/bin/env bash

#$ -l h_rt=20:00:00 -l h_vmem=20G
#$ -o /fast/AG_Ohler/manuel/splitseq/frozen_preserved_cells_nuclei_200_UBCs/output_file.txt
#$ -V
#$ -e /fast/AG_Ohler/manuel/splitseq/frozen_preserved_cells_nuclei_200_UBCs/err_file.txt
#$ -pe smp 8

HOME_Dir="/fast/AG_Ohler/manuel/splitseq"
INPUT_fastq=$HOME_Dir/frozen_preserved_cells_nuclei_200_UBCs/selected_gens.fastq
ANNOTATED_GENOME=$HOME_Dir/mus_musculus_metadata_mm10/annotated_genome

STAR \
     --runThreadN 8 \
     --genomeDir $ANNOTATED_GENOME \
     --readFilesIn $INPUT_fastq \
     --outFileNamePrefix $HOME_Dir/frozen_preserved_cells_nuclei_200_UBCs/star_aligned/star_aligned \
     --outSAMunmapped Within \
     --outFilterScoreMinOverLread 0.33 \
     --outFilterMatchNminOverLread 0.33

## Add gene name to aligned genomic reads

In [14]:
%%bash
#!/usr/bin/env bash

dropseq_tools="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix/sample_bash_scripts/drop_seq_tools"
aligned="/Users/manuel/Desktop/bowtie_strategy/star_aligned.sam"
out="/Users/manuel/Desktop/bowtie_strategy/genfun_tagged.sam"
META_DATA="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/sample_data/mm10_genome/genome_annotation/mm10.refFlat"

$dropseq_tools/TagReadWithGeneFunction \
     I=$aligned \
     O=$out \
     ANNOTATIONS_FILE=$META_DATA


INFO	2019-11-29 13:53:55	TagReadWithGeneFunction	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    TagReadWithGeneFunction -I /Users/manuel/Desktop/bowtie_strategy/star_aligned.sam -O /Users/manuel/Desktop/bowtie_strategy/genfun_tagged.sam -ANNOTATIONS_FILE /Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/sample_data/mm10_genome/genome_annotation/mm10.refFlat
**********


13:53:55.575 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix/sample_bash_scripts/drop_seq_tools/jar/lib/picard-2.18.14.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Fri Nov 29 13:53:55 CET 2019] Ta

## Extract gene names of aligned genomic reads

In [15]:
%%bash
#!/usr/bin/env bash

scripts="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix"

sam_file="/Users/manuel/Desktop/bowtie_strategy/genfun_tagged.sam"
out="/Users/manuel/Desktop/bowtie_strategy"
name="sorted_gene_names"

python3 $scripts/extract_gene_name.py \
    --sam_in $sam_file \
    --out_dir $out \
    --file_name $name \
    --accepted_gf "CODING,UTR,INTERGENIC,INTRONIC"


accepted reads: 90880
total reads: 273289


## Cluster selected CBC reads (Starcode)

In [9]:
%%bash
#!/usr/bin/env bash

inp="/Users/manuel/Desktop/bowtie_strategy/selected_barcodes.txt"
out="/Users/manuel/Desktop/bowtie_strategy/clustered_barcodes.txt"

starcode \
     --threads 4 \
     --input $inp \
     --output $out \
     -d 1 \
     -r 1 \
     --seq-id

running starcode with 4 threads
reading input files
raw format detected
sorting
progress: 100.00%
message passing clustering


## Cluster selected UMIs (Starcode)

In [10]:
%%bash
#!/usr/bin/env bash

inp="/Users/manuel/Desktop/bowtie_strategy/selected_umis.txt"
out="/Users/manuel/Desktop/bowtie_strategy/clustered_UMIs.txt"

starcode \
     --threads 4 \
     --input $inp \
     --output $out \
     -d 1 \
     -r 1 \
     --seq-id

running starcode with 4 threads
reading input files
raw format detected
sorting
progress: 100.00%
message passing clustering


## Calcuate Clusters (demultiplexing)

In [16]:
%%bash
#!/usr/bin/env bash

scripts="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix"

barcode_clusters="/Users/manuel/Desktop/bowtie_strategy/clustered_barcodes.txt"
umi_clusters="/Users/manuel/Desktop/bowtie_strategy/clustered_UMIs.txt"
genes="/Users/manuel/Desktop/bowtie_strategy/sorted_gene_names.txt"
out="/Users/manuel/Desktop/bowtie_strategy"
name="cluster_out"

python3 $scripts/calculate_clusters.py \
    --cbc_clusters $barcode_clusters \
    --umi_clusters $umi_clusters \
    --gene_names $genes \
    --out_dir $out \
    --file_name $name

## Calculate DGE matrix (UMI collapsing)

In [17]:
%%bash
#!/usr/bin/env bash

scripts="/Users/manuel/OneDrive/SPLiT-seq/SPLiT-seq_suite/DGE_matrix_generation/SPLiT-seq_DGE_matrix"

clusters="/Users/manuel/Desktop/bowtie_strategy/cluster_out.txt"
out="/Users/manuel/Desktop/bowtie_strategy"

name="expression"

python3 $scripts/calculate_DGE_matrix.py \
    --clusters $clusters \
    --out_dir $out \
    --file_name $name

number of clusters: 6115
empty clusters formed
clusters populated
umis collapsed
gene list obtained
barcode list obtained
matrix calculated
DGE matrix written to file


## A look at the produced DGE matrix

In [2]:
library(SingleCellExperiment)

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which

In [3]:
path_to_matrix = "/Users/manuel/Desktop/bowtie_strategy/expression.txt"
matrix = read.table(path_to_matrix, header=T, sep="\t")
rownames(matrix) = matrix[,1]
matrix = matrix[,-1]
matrix = data.frame(matrix)
sce_my <- SingleCellExperiment(assays = list(counts = matrix))

In [4]:
assay(sce_my)[1:10, 1:5]

,ACTATGCAACAGCAGAACTATGCA,ACATTGGCAAGAGATCACAAGCTA,ACATTGGCAAGAGATCGTCTGTCA,CTCAATGAATTGGCTCAACGTGAT,GACTAGTAACGTATCAGAATCTGA
,<int>,<int>,<int>,<int>,<int>
Hspe1,1,0,0,1,0
Gm26924,257,203,116,93,90
Csmd3,1,0,0,0,0
Glg1,1,0,0,1,0
Hnrnpa1,2,0,0,0,0
Birc6,1,1,0,1,1
Acacb,10,7,5,4,3
Idh3a,1,0,0,0,0
Cmtr2,1,0,0,0,0
